In [4]:
import csv
import re
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

# --- CONFIGURAZIONE ---
# Percorsi dei file RGB specificati
# Questi percorsi sono tipici per un ambiente Kaggle quando i dati sono stati aggiunti come dataset.
# ASSICURATI che 'datasetrgb1000' sia il nome corretto del tuo dataset su Kaggle.
FILE_RGB_CANZONI = '/kaggle/input/datasetrgb1000/valutazione_rgb_segmenti.csv'
FILE_RGB_IMMAGINI = '/kaggle/input/datasetrgb1000/valutazione_rgb_immagini1000.csv'
# Percorso del file CSV di output
OUTPUT_MAPPING_CSV = './classifica_abbinamenti_canzoni_immagini.csv' # Questo è ok, crea nella cartella di lavoro di Kaggle

# --- FUNZIONI BASE ---
def parse_rgb(rgb_str):
   
    if rgb_str is None:
        rgb_str = ''
    
    # Rimuovi spazi bianchi all'inizio e alla fine
    cleaned_str = rgb_str.strip() 

    # Ordine di priorità per il matching (dal più specifico al meno specifico)
    # Cerca 'rgb(...)' racchiuso da qualsiasi numero di virgolette triple (una o più sequenze di """""").
    match = re.search(r'(?:\"\"\")+rgb\((\d+),\s*(\d+),\s*(\d+)\)(?:\"\"\')+', cleaned_str)
    if match:
        return list(map(int, match.groups()))

    # Singola virgoletta esterna ("rgb(...)")
    match = re.search(r'"rgb\((\d+),\s*(\d+),\s*(\d+)\)"', cleaned_str)
    if match:
        return list(map(int, match.groups()))

    # Nessuna virgoletta (rgb(...)) - Fallback
    match = re.search(r'rgb\((\d+),\s*(\d+),\s*(\d+)\)', cleaned_str)
    if match:
        return list(map(int, match.groups()))
    else:
        # Questo messaggio verrà stampato solo se una stringa non è stata parsata affatto.
        if cleaned_str: 
            print(f"DEBUG parse_rgb: NO MATCH for '{cleaned_str}' - Expected an RGB format. Returning [0,0,0].")
        return [0, 0, 0]

def carica_dati_rgb(percorso, is_canzone=True):
    """
    Carica i dati RGB da un file. Utilizza un parsing manuale basato su regex
    a causa del formato non standard senza delimitatori chiari tra i campi RGB
    o per gestire le specifiche quotature complesse.
    """
    dati = {}
    try:
        with open(percorso, 'r', newline='', encoding='utf-8') as f:
            lines = f.readlines()
            
            if not lines:
                print(f"Attenzione: Il file '{os.path.basename(percorso)}' è vuoto.")
                return {}

            # La prima riga è l'header, la salta
            header_line = lines[0].strip()
            # print(f"DEBUG carica_dati_rgb: Intestazioni di '{os.path.basename(percorso)}': '{header_line}'") # Rimosso per pulizia output

            for row_idx, line in enumerate(lines[1:]): # Inizia dalla seconda riga (dati)
                line = line.strip()
                if not line: # Salta righe vuote
                    continue

                # DEBUG ATTIVO: Stampa le prime 5 righe grezze per il debug
                # if row_idx < 5: 
                #     print(f"\nDEBUG carica_dati_rgb: Analisi riga {row_idx + 1} da '{os.path.basename(percorso)}'")
                #     print(f"DEBUG carica_dati_rgb: Riga grezza: '{line}'") 

                chiave = None
                rgb_strings_found = []

                if is_canzone:
                    # Formato Canzoni: '"NomeCanzone,"""rgb1""","""rgb2"""..."'
                    # La riga intera è racchiusa in singole virgolette esterne ('"')
                    # Rimuoviamo prima queste virgolette più esterne se presenti
                    if line.startswith('"') and line.endswith('"'):
                        line = line[1:-1] # Rimuovi la prima e l'ultima virgoletta
                    
                    # Ora la riga dovrebbe essere "NomeCanzone,"""rgb1""","""rgb2"""..."
                    # Usiamo split sulla prima virgola per separare il nome del file dal resto
                    parts = line.split(',', 1) 
                    chiave = parts[0].strip() # Rimuovi spazi extra dal nome del file
                    if len(parts) > 1:
                        rgb_data_segment = parts[1]
                        # Troviamo tutte le occorrenze di '"""rgb(...)"""' o simili
                        rgb_strings_found = re.findall(r'(?:\"\"\")+rgb\(\d+,\s*\d+,\s*\d+\)(?:\"\"\')+', rgb_data_segment)
                        if not rgb_strings_found: # Fallback se il primo regex non trova, per virgolette singole o doppie
                             rgb_strings_found = re.findall(r'"rgb\(\d+,\s*\d+,\s*\d+\)"', rgb_data_segment)


                    # CHIAVE AUDIO: NON rimuovere più l'estensione .mp3
                    # La riga seguente è stata RIMOSSA per mantenere l'estensione.
                    # chiave = os.path.splitext(chiave)[0] 

                else: # Immagini
                    # Per le immagini, usiamo csv.reader su ogni riga per sfruttare la sua capacità di separare i campi
                    reader_temp = csv.reader([line]) # Trasforma la riga in un iteratore per csv.reader
                    try:
                        row_parsed = next(reader_temp)
                        chiave = row_parsed[0].strip()
                        rgb_strings_found = row_parsed[1:] # Prendi tutti i campi RGB

                    except Exception as e:
                        print(f"Attenzione (Immagini): Fallito parsing con csv.reader per riga: '{line}'. Errore: {e}. Riga saltata.")
                        continue
                
                # DEBUG ATTIVO: Mostra la chiave estratta e le stringhe RGB trovate
                # if row_idx < 5:
                #     print(f"DEBUG carica_dati_rgb: Chiave estratta: '{chiave}'")
                #     print(f"DEBUG carica_dati_rgb: Stringhe RGB trovate: {rgb_strings_found}")

                vettore_rgb_completo = []
                # Ci aspettiamo 5 valori RGB (4 frammenti + 1 totale)
                expected_rgb_count = 5 
                
                for rgb_str_item in rgb_strings_found[:expected_rgb_count]:
                    vettore_rgb_completo.extend(parse_rgb(rgb_str_item))
                
                # Assicurati che il vettore abbia 15 elementi (5 * 3)
                while len(vettore_rgb_completo) < 15:
                    vettore_rgb_completo.extend([0, 0, 0]) # Riempi con zeri se mancano valori

                if len(vettore_rgb_completo) == 15:
                    dati[chiave] = vettore_rgb_completo
                else:
                    print(f"Attenzione: Vettore RGB incompleto per '{chiave}' nel file '{os.path.basename(percorso)}'. Lunghezza: {len(vettore_rgb_completo)}. Riga saltata.")
    except FileNotFoundError:
        print(f"Errore: File non trovato al percorso '{percorso}'. Controlla che i percorsi siano corretti e i file siano nel dataset Kaggle.")
    except Exception as e:
        print(f"Errore generico durante il caricamento del file '{percorso}': {e}")
    return dati

# --- CARICAMENTO DATI ---
print("Caricamento dati RGB...")
canzoni_rgb_data = carica_dati_rgb(FILE_RGB_CANZONI, True)
immagini_rgb_data = carica_dati_rgb(FILE_RGB_IMMAGINI, False)
print(f"Caricate {len(canzoni_rgb_data)} canzoni e {len(immagini_rgb_data)} immagini.")

# Verifica se i dati sono stati caricati correttamente
if not canzoni_rgb_data:
    print("Errore critico: Nessun dato RGB caricato per le canzoni. Impossibile procedere. Controlla il file, il delimitatore e le colonne.")
    exit() 
if not immagini_rgb_data:
    print("Errore critico: Nessun dato RGB caricato per le immagini. Impossibile procedere. Controlla il file, il delimitatore e le colonne.")
    exit()

# --- CALCOLO MAPPING 1:1 ---
print("\nCalcolo abbinamenti 1:1 in ordine di similarità...")

# Preparazione dei dati per la similarità coseno
tutte_canzoni = list(canzoni_rgb_data.keys())
tutte_immagini = list(immagini_rgb_data.keys())

vettori_canzoni = np.array([canzoni_rgb_data[s] for s in tutte_canzoni])
vettori_immagini = np.array([immagini_rgb_data[i] for i in tutte_immagini])

matrice_similarita = cosine_similarity(vettori_canzoni, vettori_immagini)

tutte_coppie = []
for i, canzone in enumerate(tutte_canzoni):
    for j, immagine in enumerate(tutte_immagini):
        sim = matrice_similarita[i][j]
        tutte_coppie.append((sim, canzone, immagine))

tutte_coppie.sort(key=lambda x: x[0], reverse=True)

abbinamenti_univoci = []
immagini_assegnate = set() 
canzoni_assegnate = set()  

for sim, canzone, immagine in tutte_coppie:
    if canzone not in canzoni_assegnate and immagine not in immagini_assegnate:
        abbinamenti_univoci.append((sim, canzone, immagine))
        immagini_assegnate.add(immagine)
        canzoni_assegnate.add(canzone)
        
        if len(abbinamenti_univoci) == min(len(tutte_canzoni), len(tutte_immagini)):
            break

print(f"Trovati {len(abbinamenti_univoci)} abbinamenti univoci 1:1.")

# --- SALVATAGGIO RISULTATI CSV ---
print(f"\nSalvataggio della classifica in '{OUTPUT_MAPPING_CSV}'...")
output_dir = os.path.dirname(OUTPUT_MAPPING_CSV)
if output_dir and not os.path.exists(output_dir):
    os.makedirs(output_dir)

try:
    with open(OUTPUT_MAPPING_CSV, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Posizione', 'Similarità Coseno', 'Canzone', 'Immagine Abbinata'])
        for idx, (sim, canzone, immagine) in enumerate(abbinamenti_univoci, 1):
            writer.writerow([idx, f"{sim:.4f}", canzone, immagine])
    print("Classifica salvata con successo.")
except Exception as e:
    print(f"Errore durante il salvataggio della classifica: {e}")

# --- CALCOLO E STAMPA STATISTICHE FINALI ---
print("\n--- Statistiche Finali ---")

total_songs = len(canzoni_rgb_data) 
total_images = len(immagini_rgb_data) 
matched_pairs = len(abbinamenti_univoci)
unmatched_songs = total_songs - matched_pairs
unmatched_images = total_images - matched_pairs

similarities = [sim for sim, _, _ in abbinamenti_univoci]
mean_similarity = np.mean(similarities) if similarities else 0.0
max_similarity = np.max(similarities) if similarities else 0.0
min_similarity = np.min(similarities) if similarities else 0.0

print(f"Totale canzoni: {total_songs}")
print(f"Totale immagini: {total_images}")
print(f"Abbinamenti effettuati: {matched_pairs}")
print(f"Canzoni non abbinate: {unmatched_songs}")
print(f"Immagini non abbinate: {unmatched_images}")
print(f"Similarità media: {mean_similarity:.4f}")
print(f"Similarità massima: {max_similarity:.4f}")
print(f"Similarità minima: {min_similarity:.4f}")

print("\nProcesso di calcolo e abbinamento completato.")

Caricamento dati RGB...
Caricate 1000 canzoni e 1000 immagini.

Calcolo abbinamenti 1:1 in ordine di similarità...
Trovati 1000 abbinamenti univoci 1:1.

Salvataggio della classifica in './classifica_abbinamenti_canzoni_immagini.csv'...
Classifica salvata con successo.

--- Statistiche Finali ---
Totale canzoni: 1000
Totale immagini: 1000
Abbinamenti effettuati: 1000
Canzoni non abbinate: 0
Immagini non abbinate: 0
Similarità media: 0.9391
Similarità massima: 1.0000
Similarità minima: 0.4979

Processo di calcolo e abbinamento completato.
